Résumé de la recherche : 

- CaracSup n'a pas de codes en double dans une même ligne budget. Il y a cependant une ligne avec F en guise de caracsup (plutôt que '{"@Code": "Section", "@V": "F"}' )

- MtSup a des codes en doubles, 146 lignes, 25 fichiers sur un echantillon de 20k fichiers. Il n'y a que des float (des nombres), donc il y a possibilité de les additionner. Dans tout les cas il faut un arbitrage sur comment gérer ça : Conserver la valeur la plus élevée ? La dernière valeur ajoutée ? Les additionner ? 

- 

In [1]:
import pandas as pd 
import json 

In [2]:
echantillon = pd.read_parquet('./libelle_comptable/parquet/echantillon_2020.parquet')
df = echantillon[['Id_Fichier','MtSup',
       'MtSup_APVote', 'MtSup_Brut', 'MtSup_BudgetHorsRAR', 'MtSup_Comp',
       'MtSup_ICNE', 'MtSup_ICNEPrec', 'MtSup_MtOpeCumul', 'MtSup_MtOpeInfo',
       'MtSup_Net', 'MtSup_ProdChaRat', 'MtSup_RARPrec', 'CaracSup',
       'CaracSup_TypOpe', 'CaracSup_Section', 'CaracSup_ChapSpe',
       'CaracSup_ProgAutoLib', 'CaracSup_ProgAutoNum', 'CaracSup_VirCredNum',
       'CaracSup_CodeRegion']]

In [3]:
len(echantillon)

998272

In [4]:
ll = df[~df['MtSup'].isna()]['MtSup'].to_list()
ll[0]

'{"@Code": "BudgetHorsRAR", "@V": "9800"}'

In [5]:
liste_mtsup = []
liste_blabla = []
for i in ll : 
  try :
   liste_mtsup.append(json.loads(i))
  except json.JSONDecodeError as e:
   liste_blabla.append(i)

l_mt = []
for i in liste_mtsup : 
  if isinstance(i, list) : 
    l_mt.append(i)

In [6]:
liste_blabla

[]

Voici les fichiers concernés par le problème de caracsup. 

In [7]:
echantillon[echantillon['CaracSup'] == 'F']

,id,Id_Fichier,Nomenclature,Exer,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,...,MtSup_ProdChaRat,MtSup_RARPrec,CaracSup,CaracSup_TypOpe,CaracSup_Section,CaracSup_ChapSpe,CaracSup_ProgAutoLib,CaracSup_ProgAutoNum,CaracSup_VirCredNum,CaracSup_CodeRegion
239062,434080,682109,M14-M14_COM_500_3500,2020,None,None,7588,None,None,None,...,None,None,F,NaN,None,None,None,None,None,NaN
509589,1089428,756300,M14-M14_COM_500_3500,2020,None,None,752,None,None,None,...,None,None,F,NaN,None,None,None,None,None,NaN


In [8]:
df.columns

Index(['Id_Fichier', 'MtSup', 'MtSup_APVote', 'MtSup_Brut',
       'MtSup_BudgetHorsRAR', 'MtSup_Comp', 'MtSup_ICNE', 'MtSup_ICNEPrec',
       'MtSup_MtOpeCumul', 'MtSup_MtOpeInfo', 'MtSup_Net', 'MtSup_ProdChaRat',
       'MtSup_RARPrec', 'CaracSup', 'CaracSup_TypOpe', 'CaracSup_Section',
       'CaracSup_ChapSpe', 'CaracSup_ProgAutoLib', 'CaracSup_ProgAutoNum',
       'CaracSup_VirCredNum', 'CaracSup_CodeRegion'],
      dtype='object')

In [9]:
l_mt[20]

[{'@Code': 'MtOpeCumul', '@V': '106960.33'},
 {'@Code': 'BudgetHorsRAR', '@V': '1500'}]

In [10]:
nb = 0
liste_duplic = []
for i in l_mt : 
  nb += 1
  code_counts = {}
  for j in i : 
    code = j.get('@Code')
    if code in code_counts :
      code_counts[code] += 1
    else :
      code_counts[code] = 1
  duplicates = [code for code, count in code_counts.items() if count > 1]
  if duplicates : 
    liste_duplic.append(i)

Voici un exemple de MtSup contenant plusieurs fois le même code. 

- Par défaut, python va sélectionner le dernier, dans notre cas 75000.00 

In [11]:
liste_duplic[0]

[{'@Code': 'ProdChaRat', '@V': '127797.54'},
 {'@Code': 'BudgetHorsRAR', '@V': '5408000.00'},
 {'@Code': 'BudgetHorsRAR', '@V': '75000.00'}]

In [12]:
len(liste_duplic)

142

In [13]:
df[(df['MtSup_BudgetHorsRAR'] == 75000.00) & (df['MtSup_ProdChaRat'] == '127797.54')]

,Id_Fichier,MtSup,MtSup_APVote,MtSup_Brut,MtSup_BudgetHorsRAR,MtSup_Comp,MtSup_ICNE,MtSup_ICNEPrec,MtSup_MtOpeCumul,MtSup_MtOpeInfo,...,MtSup_ProdChaRat,MtSup_RARPrec,CaracSup,CaracSup_TypOpe,CaracSup_Section,CaracSup_ChapSpe,CaracSup_ProgAutoLib,CaracSup_ProgAutoNum,CaracSup_VirCredNum,CaracSup_CodeRegion
2140,761216,"[{""@Code"": ""ProdChaRat"", ""@V"": ""127797.54""}, {...",NaN,NaN,75000.0,None,NaN,NaN,NaN,NaN,...,127797.54,None,None,NaN,None,None,None,None,None,NaN


In [14]:
str(liste_duplic[0])

"[{'@Code': 'ProdChaRat', '@V': '127797.54'}, {'@Code': 'BudgetHorsRAR', '@V': '5408000.00'}, {'@Code': 'BudgetHorsRAR', '@V': '75000.00'}]"

In [15]:
liste_duplic_propre = []
for i in liste_duplic : 
  dict_tot = {}
  for l in i :
   dict_tot.update({l.get('@Code') : l.get('@V')})
  liste_duplic_propre.append(dict_tot)

In [16]:
liste_duplic_propre[0]

{'ProdChaRat': '127797.54', 'BudgetHorsRAR': '75000.00'}

In [17]:
df.dtypes 

Id_Fichier               object
MtSup                    object
MtSup_APVote            float64
MtSup_Brut              float64
MtSup_BudgetHorsRAR     float64
MtSup_Comp               object
MtSup_ICNE              float64
MtSup_ICNEPrec          float64
MtSup_MtOpeCumul        float64
MtSup_MtOpeInfo         float64
MtSup_Net               float64
MtSup_ProdChaRat         object
MtSup_RARPrec            object
CaracSup                 object
CaracSup_TypOpe         float64
CaracSup_Section         object
CaracSup_ChapSpe         object
CaracSup_ProgAutoLib     object
CaracSup_ProgAutoNum     object
CaracSup_VirCredNum      object
CaracSup_CodeRegion     float64
dtype: object

In [18]:
df_duplic = pd.DataFrame(liste_duplic_propre)
df_duplic['BudgetHorsRAR'] = df_duplic['BudgetHorsRAR'].astype(float)
df_duplic['MtOpeCumul'] = df_duplic['MtOpeCumul'].astype(float)
df_duplic['ICNE'] = df_duplic['ICNE'].astype(float)
df_duplic['ICNEPrec'] = df_duplic['ICNEPrec'].astype(float)
df_duplic.dtypes 

ProdChaRat        object
BudgetHorsRAR    float64
MtOpeCumul       float64
ICNE             float64
ICNEPrec         float64
dtype: object

In [19]:
df_duplic_et_id = df_duplic.merge(echantillon, left_on = ['ProdChaRat','BudgetHorsRAR', 'MtOpeCumul', 'ICNE', 'ICNEPrec'], 
                              right_on= ['MtSup_ProdChaRat', 'MtSup_BudgetHorsRAR','MtSup_MtOpeCumul', 'MtSup_ICNE', 'MtSup_ICNEPrec'], how = 'left')
liste_id = df_duplic_et_id['Id_Fichier'].drop_duplicates().to_list()
print(len(df_duplic_et_id))
print(len(liste_id))

146
25


In [35]:
echantillon[ (echantillon['Id_Fichier'] == '761216')
        &(echantillon['MtSup_BudgetHorsRAR'] == 75000)
        & (echantillon['MtSup_ProdChaRat'] == '127797.54' )]['MtSup'].to_list()

['[{"@Code": "ProdChaRat", "@V": "127797.54"}, {"@Code": "BudgetHorsRAR", "@V": "5408000.00"}, {"@Code": "BudgetHorsRAR", "@V": "75000.00"}]']

In [32]:
echantillon[echantillon['Id_Fichier'] == '761216']['MtSup']

,id,Id_Fichier,Nomenclature,Exer,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,...,MtSup_ProdChaRat,MtSup_RARPrec,CaracSup,CaracSup_TypOpe,CaracSup_Section,CaracSup_ChapSpe,CaracSup_ProgAutoLib,CaracSup_ProgAutoNum,CaracSup_VirCredNum,CaracSup_CodeRegion
2111,2337587,761216,M4-M43_D,2020,None,None,66112,None,None,None,...,6640.44,None,None,NaN,None,None,None,None,None,NaN
2112,2337588,761216,M4-M43_D,2020,None,None,648,None,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN
2113,2337589,761216,M4-M43_D,2020,None,None,658,None,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN
2114,2337590,761216,M4-M43_D,2020,None,None,66111,None,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN
2115,2337591,761216,M4-M43_D,2020,None,None,678,None,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN
2116,2337592,761216,M4-M43_D,2020,Opération de section à section,None,6811,042,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN
2117,2337593,761216,M4-M43_D,2020,None,None,7087,None,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN
2118,2337594,761216,M4-M43_D,2020,None,None,734,None,None,None,...,9700.10,None,None,NaN,None,None,None,None,None,NaN
2119,2337595,761216,M4-M43_D,2020,None,None,7473,None,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN
2120,2337596,761216,M4-M43_D,2020,None,None,7475,None,None,None,...,None,None,None,NaN,None,None,None,None,None,NaN


In [20]:
df_duplic[df_duplic['BudgetHorsRAR'] == 75000]

,ProdChaRat,BudgetHorsRAR,MtOpeCumul,ICNE,ICNEPrec
0,127797.54,75000.0,NaN,NaN,NaN


In [38]:
df_duplic_et_id.columns 

df_duplic_et_id = df_duplic_et_id[['Id_Fichier', 'Nomenclature','Exer','TypOpBudg','Operation',
        'Nature','ContNat','LibCpte', 'Fonction', 'ContFon', 'ArtSpe',
       'CodRD', 'MtBudgPrec', 'MtRARPrec', 'MtPropNouv', 'MtPrev', 'OpBudg',
       'CredOuv', 'MtReal', 'MtRAR3112', 'ContOp', 'OpeCpteTiers', 
       'ProdChaRat', 'BudgetHorsRAR', 'MtOpeCumul', 'ICNE', 'ICNEPrec']]
df_duplic_et_id

,Id_Fichier,Nomenclature,Exer,TypOpBudg,Operation,Nature,ContNat,LibCpte,Fonction,ContFon,...,CredOuv,MtReal,MtRAR3112,ContOp,OpeCpteTiers,ProdChaRat,BudgetHorsRAR,MtOpeCumul,ICNE,ICNEPrec
0,761216,M4-M43_D,2020,None,None,611,None,None,None,None,...,5483000.00,5440331.26,0.0,None,None,127797.54,75000.00,NaN,NaN,NaN
1,705047,M14-M14_COM_SUP3500,2020,None,None,60612,None,None,01,None,...,76020.00,46226.59,0.0,None,None,3995.19,67520.00,NaN,NaN,NaN
2,705047,M14-M14_COM_SUP3500,2020,None,None,60632,None,None,810,None,...,66500.00,84777.46,0.0,None,None,1173.47,6500.00,NaN,NaN,NaN
3,705047,M14-M14_COM_SUP3500,2020,None,None,60632,None,None,213,None,...,15500.00,19683.05,0.0,None,None,1192.04,12000.00,NaN,NaN,NaN
4,705047,M14-M14_COM_SUP3500,2020,None,None,6067,None,None,213,None,...,35570.00,31011.71,0.0,None,None,745.82,300.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,716109,M14-M14_COM_SUP3500,2020,None,None,6064,None,None,211,None,...,1203.24,1299.96,0.0,None,None,175.34,480.50,NaN,NaN,NaN
142,716109,M14-M14_COM_SUP3500,2020,None,None,6067,None,None,212,None,...,29795.31,27826.25,0.0,None,None,1781.43,1265.00,NaN,NaN,NaN
143,716109,M14-M14_COM_SUP3500,2020,None,None,6067,None,None,211,None,...,13879.44,13337.70,0.0,None,None,1598.34,2514.69,NaN,NaN,NaN
144,716109,M14-M14_COM_SUP3500,2020,None,None,6238,None,None,023,None,...,7050.00,3334.88,0.0,None,None,66.00,5119.00,NaN,NaN,NaN


In [39]:
df_duplic_et_id[df_duplic_et_id['Id_Fichier'] == '716109']['BudgetHorsRAR']

132    103015.00
133      5500.00
134       500.00
135       926.96
136      6051.16
137     12428.10
138      1060.00
139      1838.77
140        75.71
141       480.50
142      1265.00
143      2514.69
144      5119.00
145      1518.44
Name: BudgetHorsRAR, dtype: float64

In [49]:
echantillon[(echantillon['Id_Fichier'] == "716109") & (~echantillon['MtSup'].isna())]['MtSup'].to_list()[40:100]

['{"@Code": "BudgetHorsRAR", "@V": "690052.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "19000.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "7000.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "600.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "500.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "480.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "2465.38"}',
 '{"@Code": "BudgetHorsRAR", "@V": "1000.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "37550.00"}',
 '[{"@Code": "BudgetHorsRAR", "@V": "5500.00"}, {"@Code": "ProdChaRat", "@V": "450000.00"}, {"@Code": "ProdChaRat", "@V": "35000.00"}]',
 '{"@Code": "BudgetHorsRAR", "@V": "312.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "59256.21"}',
 '{"@Code": "BudgetHorsRAR", "@V": "28.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "183022.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "0.56"}',
 '[{"@Code": "ProdChaRat", "@V": "444.00"}, {"@Code": "BudgetHorsRAR", "@V": "1586.00"}]',
 '{"@Code": "BudgetHorsRAR", "@V": "500.00"}',
 '{"@Code": "BudgetHorsRAR", "@V": "500.00"}',
 '{"@Cod

In [22]:
df_duplic_et_id.to_csv('./recette_mtsup_caracsup.csv')

In [23]:
len(df_duplic_et_id['Id_Fichier'].drop_duplicates().to_list())

25

In [24]:
df_duplic_et_id.columns

Index(['Id_Fichier', 'Nomenclature', 'Exer', 'TypOpBudg', 'Operation',
       'Nature', 'ContNat', 'LibCpte', 'Fonction', 'ContFon', 'ArtSpe',
       'CodRD', 'MtBudgPrec', 'MtRARPrec', 'MtPropNouv', 'MtPrev', 'OpBudg',
       'CredOuv', 'MtReal', 'MtRAR3112', 'ContOp', 'OpeCpteTiers',
       'ProdChaRat', 'BudgetHorsRAR', 'MtOpeCumul', 'ICNE', 'ICNEPrec'],
      dtype='object')

In [25]:
df_duplic_et_id['Nomenclature'].value_counts()

Nomenclature
M14-M14_COM_SUP3500     117
M4-M49_D                 13
M61-M61                   8
M52-M52                   3
M4-M43_D                  2
M14-M14_CCAS_SUP3500      2
M4-M49_A                  1
Name: count, dtype: int64